In [18]:
!pip3 install lxml

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from platform import python_version
import os, time, socket
import pandas as pd
import bs4, lxml
from bs4 import BeautifulSoup
from lxml import etree
import requests

In [2]:
def day_price(result1) :
    #股價及其他資訊
    df = pd.DataFrame(result1[3])
    return df

In [3]:
def people(result0) :#股價持股等級
    df1 = pd.DataFrame(result0[23]) #[0]法人買賣情況(買進買出)
    df2 = pd.DataFrame(result0[30]) #[1]股東持有者
    df3 = pd.DataFrame(result0[32]) #[2]法人買賣情況(本國個人、法人)
    df4 = pd.DataFrame(result0[34]) #[3]持股分級前五名
    
    return df1,df2,df3,df4

In [4]:
def percent(result2) :
    #股東持股比例，以loc留住特定列，在'持有比例.7'後無資料，故留到'持有比例.7'
    df5 = pd.DataFrame(result2[18].loc[:13,['週別', '統計日期','當週股價','當週股價.1','當週股價.2','集保庫存(萬張)','各持股等級股東之持有比例(%)',
                                       '各持股等級股東之持有比例(%).1',
                                       '各持股等級股東之持有比例(%).2',
                                       '各持股等級股東之持有比例(%).3',
                                       '各持股等級股東之持有比例(%).4',
                                       '各持股等級股東之持有比例(%).5',
                                       '各持股等級股東之持有比例(%).6',
                                       '各持股等級股東之持有比例(%).7',]])
    return df5

In [11]:
def stocks(stock):
    url = f"https://goodinfo.tw/tw/StockDetail.asp?STOCK_ID={stock}" #url_1，有股價及其他資訊
    payloads = {"STOCK_ID": f"{stock}"}
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

    resp = requests.request(method="POST", url=url, data=payloads, headers=headers)
    resp.encoding = "utf-8"
    result1 = pd.read_html(resp.text, header=1) #股價表需要設header=1，表頭才會是以下呈現樣式
    result0 = pd.read_html(resp.text, header=0) #法人買賣等表需要設header=0，表頭才會是以下呈現樣式
    
    url = f"https://goodinfo.tw/tw/EquityDistributionClassHis.asp?STOCK_ID={stock}" #url_2，股東持股比例
    payloads = {"STOCK_ID": f"{stock}"}
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

    resp = requests.request(method="POST", url=url, data=payloads, headers=headers)
    resp.encoding="utf-8"

    result2 = pd.read_html(resp.text, header=0) #持股比例表表需要設header=0，表頭才會是以下呈現樣式
    
    return day_price(result1),people(result0),percent(result2) #分別設到[0]day_price(股價表)，people[1](買進買出股東)，percent[2](持股比例)

In [12]:
stocks(4558)[0].head()

,成交價,昨收,漲跌價,漲跌幅,振幅,開盤,最高,最低
0,23.75,23.75,0,0%,2.53%,24.35,24.35,23.75
1,成交張數,成交金額,成交筆數,成交均張,成交均價,PBR,PER,PEG
2,32,76.82 萬,27,1.2 張/筆,24.01 元,1.3,11.31,-0.23
3,昨日張數,昨日金額,昨日筆數,昨日均張,昨日均價,昨漲跌價 (幅),昨漲跌價 (幅),昨漲跌價 (幅)
4,35,83.74 萬,33,1.1 張/筆,23.92 元,+0.15 (+0.64%),+0.15 (+0.64%),+0.15 (+0.64%)


In [61]:
stocks(4558)[1][0].head()

,Unnamed: 0,買進(張),賣出(張),買賣超,連買連賣,持有張數,持股比
0,外資,5,1,4,賣→買,43.0,0.08%
1,投信,0,0,0,-,NaN,NaN
2,自營商,0,0,0,-,NaN,NaN
3,合計,5,1,4,賣→買,NaN,NaN


In [22]:
stocks(4558)[1][1].head()

,持股人,持股張數,持股比例,本月增減,質押張數,質押比例
0,非獨立董監,5231,10.5%,0,0,0%
1,獨立董監,0,0%,0,0,0%
2,全體董監,5231,10.5%,0,0,0%


In [64]:
stocks(4558)[1][2].head()

,持有股數排名,人數,持有股數(千股),人均持股(千股),持股比例(%)
0,本國個人,326,37724,116,85.54
1,本國法人,10,6376,638,14.46


In [14]:
stocks(4558)[1][3].head()

,持股分級前五名(股),人數,持有股數(千股),人均持股(千股),持股比例(%)
0,"1,000,001以上",11,23575,2143,47.15
1,"400,001-600,000",12,5920,493,11.84
2,"800,001-1,000,000",5,4772,954,9.54
3,"600,001-800,000",6,4054,676,8.11
4,"200,001-400,000",9,2719,302,5.44


In [13]:
stocks(4558)[2].head()

,週別,統計日期,當週股價,當週股價.1,當週股價.2,集保庫存(萬張),各持股等級股東之持有比例(%),各持股等級股東之持有比例(%).1,各持股等級股東之持有比例(%).2,各持股等級股東之持有比例(%).3,各持股等級股東之持有比例(%).4,各持股等級股東之持有比例(%).5,各持股等級股東之持有比例(%).6,各持股等級股東之持有比例(%).7
0,週別,統計日期,收盤,漲跌(元),漲跌(%),集保庫存(萬張),≦10張,＞10張≦50張,＞50張≦100張,＞100張≦200張,＞200張≦400張,＞400張≦800張,＞800張≦1千張,＞1千張
1,22W50,-,23.75,+0.65,+2.81,-,-,-,-,-,-,-,-,-
2,22W49,12/02,23.1,+0.05,+0.22,5,6.32,4.97,3.63,3,5.44,19.9,9.54,47.2
3,22W48,11/25,23.05,-0.05,-0.22,5,6.25,5.02,3.63,3,5.44,19.9,9.54,47.2
4,22W47,11/18,23.1,+0.4,+1.76,5,6.25,4.91,3.73,3,5.44,19.9,9.54,47.2


In [23]:
stocks(2352)[0].head()

,成交價,昨收,漲跌價,漲跌幅,振幅,開盤,最高,最低
0,27.2,27.35,-0.15,-0.55%,1.46%,27.3,27.5,27.1
1,成交張數,成交金額,成交筆數,成交均張,成交均價,PBR,PER,PEG
2,3086,"8,398 萬",1781,1.7 張/筆,27.21 元,1.3,6.02,-2.76
3,昨日張數,昨日金額,昨日筆數,昨日均張,昨日均價,昨漲跌價 (幅),昨漲跌價 (幅),昨漲跌價 (幅)
4,6679,1.84 億,2922,2.3 張/筆,27.51 元,-0.05 (-0.18%),-0.05 (-0.18%),-0.05 (-0.18%)
